Parâmetros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_catalag_schema

Regras e Definições

In [0]:
%sql
select * from tiller.silver.tb_despesas_silver;

In [0]:
set_catalog_and_schema_gold() #Define catalog e schema conforme run

In [0]:
query = f"""
SELECT
    d.nuDeputadoId        AS id_deputado,
    d.sgPartido           AS sigla_partido,
    d.sgUF                AS sigla_uf,
    d.ideDocumento        AS id_despesa,
    d.txtCNPJCPF          AS cnpj_cpf_fornecedor,
    d.indTipoDocumento    AS tipo_documento,
    d.txtNumero           AS numero_documento,
    d.data_emissao,
    d.valor_documento,
    d.valor_glosa,
    d.valor_liquido,
    d.urlDocumento        AS url_documento,
    d.numSubCota,
    d.numParcela,
    d.ano,
    d.mes,
    d.fonte_arquivo,
    date_trunc('second', CURRENT_TIMESTAMP()) AS data_ingestao, -- Atualiza data de ingestão até o segundo apenas date_trunc
    date_trunc('second', CURRENT_TIMESTAMP()) AS data_modificacao
FROM tiller.silver.tb_despesas_silver d
LEFT JOIN tiller.gold.dim_deputados dep
    ON CAST(d.nuDeputadoId AS STRING) = dep.id_deputado
"""

df_despesas = spark.sql(query)

df_despesas.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("fato_gastos_deputados")

print("Fato criada/atualizada com sucesso!")


Comentários das colunas

In [0]:
%sql
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.id_deputado IS 'ID do deputado conforme base fonte, referenciando dim_deputados';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.sigla_partido IS 'Sigla do partido do deputado na data da despesa';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.sigla_uf IS 'Sigla da unidade federativa do deputado na data da despesa';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.id_despesa IS 'ID da despesa conforme sistema fonte, referenciando dim_despesas';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.cnpj_cpf_fornecedor IS 'CNPJ ou CPF do fornecedor, referenciando dim_fornecedores';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.tipo_documento IS 'Tipo de documento da despesa';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.numero_documento IS 'Número do documento da despesa';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.data_emissao IS 'Data de emissão da despesa';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.valor_documento IS 'Valor total da despesa';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.valor_glosa IS 'Valor glosado da despesa';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.valor_liquido IS 'Valor líquido da despesa';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.url_documento IS 'URL do documento da despesa';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.numSubCota IS 'Número da subcota utilizada na despesa';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.numParcela IS 'Número da parcela da despesa';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.ano IS 'Ano da despesa';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.mes IS 'Mês da despesa';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.fonte_arquivo IS 'Fonte do arquivo original da despesa';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.data_ingestao IS 'Data e hora da ingestão do registro na tabela fato';
COMMENT ON COLUMN tiller.gold.fato_gastos_deputados.data_modificacao IS 'Data e hora da última modificação registrada na tabela fato';


In [0]:
%sql

SELECT COUNT(*) AS total, MIN(data_ingestao) AS primeira_carga
FROM fato_gastos_deputados;


In [0]:
%sql
select * from tiller.gold.fato_gastos_deputados;